In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

: 

In [ ]:
df = pd.read_csv(r'C:\Users\Dell\Desktop\DataSets\Walmart_Sales.csv')

In [ ]:
df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)

In [ ]:
df.isna().sum()
df.duplicated().sum()

In [ ]:
df.corr()

In [ ]:
df.drop(['Date', 'Store'],axis=1, inplace=True)

In [ ]:
corr = df.corr()
sns.heatmap(corr)

In [ ]:
df.corr()

In [ ]:
df['Holiday_Flag'].value_counts()

In [ ]:
df.groupby('Holiday_Flag')['Weekly_Sales'].mean()

In [ ]:
df['holiday_impact'] = df['Weekly_Sales']*df['Weekly_Sales']

In [ ]:
df.corr()

In [ ]:
sns.pairplot(df)

In [ ]:
fif = df.sample(100)

In [ ]:
fif.columns

In [ ]:
plt.scatter(data=fif, x='Weekly_Sales', y='Unemployment')
plt.xlabel('Weekly Sales')
plt.ylabel('Fuel_Price')
plt.show()

In [ ]:
df.corr()['Weekly_Sales']['Unemployment']

In [ ]:
df.drop('Fuel_Price',axis=1, inplace=True)
df.drop('CPI',axis=1, inplace=True)
df.drop('Temperature',axis=1, inplace=True)

In [ ]:
df

In [ ]:
sns.pairplot(df)

In [ ]:
x = df[['holiday_impact']]

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_fi = poly.fit_transform(x)
fea = poly.get_feature_names_out(x.columns)
df1 = pd.DataFrame(poly_fi, columns=fea)

In [ ]:
df.drop('holiday_impact',axis=1, inplace=True)

In [ ]:
df = pd.concat([df, df1],axis=1)

In [ ]:
df.duplicated().sum()

In [ ]:
sns.pairplot(df)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['Weekly_Sales'] = scaler.fit_transform(df[['Weekly_Sales']])

In [ ]:
df

In [ ]:
x = df.drop('Weekly_Sales',axis=1)
y = df['Weekly_Sales']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge, Lasso


x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.30, random_state=1)

model = Ridge()
param_dist = {'alpha':np.linspace(0.01, 100, 100)}

random_search = RandomizedSearchCV(
    estimator=model, param_distributions=param_dist,n_iter=10,
    scoring='neg_mean_squared_error',cv=5, random_state=1, n_jobs=-1
    )

random_search.fit(x_train, y_train)
pred = random_search.best_estimator_.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

print("Best Alpha:", random_search.best_params_['alpha'])
print(f'MSE : {mean_squared_error(y_test, pred)}')
print(f'r2_score : {r2_score(y_test, pred)}')

In [ ]:
ridge = Ridge(alpha=93.94)
ridge.fit(x_train, y_train)
y_pred = ridge.predict(x_test)

print(f'MSE : {mean_squared_error(y_test, y_pred)}')
print(f'r2_score : {r2_score(y_test, y_pred)}')

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x=y_test, y=pred, color='blue', label='Predicted VS Actual')
sns.lineplot(x=y_test, y=y_test, color='red', label='Perfect Prediction Line')
plt.xlabel('Actual Sales')

plt.ylabel('Predicted Sales')
plt.title('Linear Regression: Actual vs Predictor')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
residuals = y_test-pred
sns.displot(residuals, kind='kde')

In [ ]:
import pickle
pickle.dump(ridge, open('model.pkl','wb'))

In [ ]:
model = pickle.load(open('model.pkl', 'rb'))

In [ ]:
model.predict(x_test)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(x_train)

In [ ]:
pickle.dump(scaler, open('scaler.pkl','wb'))

In [ ]:
scaler = pickle.load(open('scaler.pkl', 'rb'))